### Guided Project #8
##### Answering Business Questions using SQL

#### Chinook DB Schema
<img src="chinook-schema.svg" width="800" height="400">

### Setting up SQLite

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT * FROM employee LIMIT 3

 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com


### Listing DB Tables


In [3]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


### 1. Genres popularity in USA
Number of tracks for each genre sold in USA

In [4]:
%%sql
SELECT 
    g.name AS genre_name, 
    SUM(il.quantity) AS tracks_sold,
    ROUND(SUM(il.quantity) * 100.0 / total_sold.sum_quantity, 2) || ' %' AS tracs_sold_pct
        FROM invoice_line il
            LEFT JOIN invoice i ON il.invoice_id = i.invoice_id
            LEFT JOIN track t ON t.track_id = il.track_id
            LEFT JOIN genre g on t.genre_id = g.genre_id
            LEFT JOIN (SELECT SUM(quantity) as sum_quantity FROM invoice_line) total_sold
    GROUP BY 1
    ORDER BY 2 DESC

 * sqlite:///chinook.db
Done.


genre_name,tracks_sold,tracs_sold_pct
Rock,2635,55.39 %
Metal,619,13.01 %
Alternative & Punk,492,10.34 %
Latin,167,3.51 %
R&B/Soul,159,3.34 %
Blues,124,2.61 %
Jazz,121,2.54 %
Alternative,117,2.46 %
Easy Listening,74,1.56 %
Pop,63,1.32 %


### 2. Sales assigned to each sales support agent
Total dollar amount of sales assigned to each sales support agent within the company.

In [5]:
%%sql
SELECT 
    MAX(e.first_name || ' ' || e.last_name) employee_name,
    MAX(e.title) title,
    MAX(e.hire_date) hire_date,
    MAX(e.city || ', ' || e.country) location,
    ROUND(SUM(i.total),2) AS total_sales
    FROM invoice i 
        LEFT JOIN customer c ON c.customer_id = i.customer_id
        LEFT JOIN employee e ON e.employee_id = c.support_rep_id
    GROUP BY e.employee_id
    ORDER BY total_sales DESC

 * sqlite:///chinook.db
Done.


employee_name,title,hire_date,location,total_sales
Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,"Calgary, Canada",1731.51
Margaret Park,Sales Support Agent,2017-05-03 00:00:00,"Calgary, Canada",1584.0
Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,"Calgary, Canada",1393.92


Let's double-check we only have 3 sales represantatives

In [6]:
%%sql
SELECT * FROM employee WHERE title = 'Sales Support Agent'

 * sqlite:///chinook.db
Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


This confirmes our result. Apparently, hire date correlates with total sales amount, which makes sense. The longer the epmloyee works, the more sales they achieve

### 3. Sales data for customers from each different country
Write a query that collates data on purchases from different countries.
Where a country has only one customer, collect them into an "Other" group.

The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.

For each country, include:
- total number of customers
- total value of sales
- average value of sales per customer
- average order value

In [7]:
%%sql
-- Preaggregation: calculating required field - intermediary result
SELECT 
    c.country,
    CASE WHEN COUNT(DISTINCT c.customer_id) = 1 THEN 'Other' ELSE c.country END group_country,
    CASE WHEN COUNT(DISTINCT c.customer_id) = 1 THEN 1 ELSE 0 END group_order,
    COUNT(DISTINCT c.customer_id) total_customers,
    COUNT(invoice_id) total_orders,
    ROUND(SUM(i.total),2) total_sales,
    ROUND(SUM(i.total)/COUNT(DISTINCT c.customer_id),2) average_sales,
    ROUND(SUM(i.total)/ COUNT(invoice_id), 2) average_order
    FROM invoice i
        LEFT JOIN customer c ON c.customer_id = i.customer_id
        
    GROUP BY c.country
    ORDER BY total_sales DESC

 * sqlite:///chinook.db
Done.


country,group_country,group_order,total_customers,total_orders,total_sales,average_sales,average_order
USA,USA,0,13,131,1040.49,80.04,7.94
Canada,Canada,0,8,76,535.59,66.95,7.05
Brazil,Brazil,0,5,61,427.68,85.54,7.01
France,France,0,5,50,389.07,77.81,7.78
Germany,Germany,0,4,41,334.62,83.66,8.16
Czech Republic,Czech Republic,0,2,30,273.24,136.62,9.11
United Kingdom,United Kingdom,0,3,28,245.52,81.84,8.77
Portugal,Portugal,0,2,29,185.13,92.57,6.38
India,India,0,2,21,183.15,91.58,8.72
Ireland,Other,1,1,13,114.84,114.84,8.83


In [8]:
%%sql
WITH pre_agg AS
(SELECT 
    c.country,
    CASE WHEN COUNT(DISTINCT c.customer_id) = 1 THEN 'Other' ELSE c.country END group_country,
    CASE WHEN COUNT(DISTINCT c.customer_id) = 1 THEN 1 ELSE 0 END group_order,
    COUNT(DISTINCT c.customer_id) total_customers,
    COUNT(invoice_id) total_orders,
    ROUND(SUM(i.total),2) total_sales,
    ROUND(SUM(i.total)/COUNT(DISTINCT c.customer_id),2) average_sales,
    ROUND(SUM(i.total)/COUNT(invoice_id), 2) average_order
        FROM invoice i
        LEFT JOIN customer c ON c.customer_id = i.customer_id
        
    GROUP BY c.country
    ORDER BY total_sales DESC
)

SELECT 
    group_country country,
    SUM(total_customers) total_customers, 
    ROUND(SUM(total_sales),2) total_sales,
    ROUND(SUM(total_sales)/SUM(total_customers),2) average_sale,
    ROUND(SUM(total_sales)/SUM(total_orders), 2) average_order
    
    FROM pre_agg
    GROUP BY group_country
    ORDER BY group_order, total_sales DESC

 * sqlite:///chinook.db
Done.


country,total_customers,total_sales,average_sale,average_order
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Other,15,1094.94,73.0,7.45


### 4. Full album vs individual track sales
Write a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:
- Number of invoices
- Percentage of invoices

#### Strating with helper queries

In [21]:
%%sql
-- Album tracks
SELECT album_id, track_id 
    FROM track
    LIMIT 3

 * sqlite:///chinook.db
Done.


album_id,track_id
1,1
1,6
1,7


In [15]:
%%sql
-- Invoice tracks
SELECT i.invoice_id, t.track_id
    FROM invoice i
    LEFT JOIN invoice_line il ON il.invoice_id = i.invoice_id
    LEFT JOIN track t ON t.track_id = il.track_id
    LIMIT 3

 * sqlite:///chinook.db
Done.


invoice_id,track_id
16,2160
16,875
16,83


In [19]:
%%sql
-- Invoice track album
-- The album of some track of invoice
SELECT i.invoice_id, MIN(t.album_id) album_id
    FROM invoice i
    LEFT JOIN invoice_line il ON il.invoice_id = i.invoice_id
    LEFT JOIN track t ON t.track_id = il.track_id
    GROUP BY 1
    LIMIT 3

 * sqlite:///chinook.db
Done.


invoice_id,album_id
1,91
2,20
3,203


#### Putting the query together

In [34]:
%%sql
WITH 
album_tracks AS (
    SELECT album_id, track_id 
    FROM track
    ),
invoice_tracks AS (
    SELECT i.invoice_id, t.track_id
    FROM invoice i
    LEFT JOIN invoice_line il ON il.invoice_id = i.invoice_id
    LEFT JOIN track t ON t.track_id = il.track_id
    ),
invoice_track_album AS (
    SELECT i.invoice_id, MIN(t.album_id) album_id
    FROM invoice i
    LEFT JOIN invoice_line il ON il.invoice_id = i.invoice_id
    LEFT JOIN track t ON t.track_id = il.track_id
    GROUP BY 1  
    )

SELECT i.invoice_id,
    ita.album_id,
    CASE WHEN (
        SELECT track_id FROM album_tracks 
            WHERE album_id = ita.album_id 
        EXCEPT
        SELECT track_id FROM invoice_tracks
            WHERE invoice_id = i.invoice_id
        ) IS NULL AND
        (
        SELECT track_id FROM invoice_tracks
            WHERE invoice_id = i.invoice_id
        EXCEPT
        SELECT track_id FROM album_tracks 
            WHERE album_id = ita.album_id 
        ) IS NULL
        THEN 'Y'
        ELSE 'N' 
        END as full_album_purchase_ind
    
    FROM invoice i
        LEFT JOIN invoice_track_album ita ON ita.invoice_id = i.invoice_id
    WHERE full_album_purchase_ind = 'Y'
    LIMIT 3

 * sqlite:///chinook.db
Done.


invoice_id,album_id,full_album_purchase_ind
153,196,Y
184,278,Y
223,213,Y


In [40]:
%%sql
-- verifying the above is correct for invoice 153
SELECT il.*, t.album_id FROM invoice_line il
    LEFT JOIN track t ON t.track_id = il.track_id
    WHERE il.invoice_id = 153

 * sqlite:///chinook.db
Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity,album_id
1198,153,2406,0.99,1,196
1199,153,2407,0.99,1,196
1200,153,2408,0.99,1,196
1201,153,2409,0.99,1,196
1202,153,2410,0.99,1,196
1203,153,2411,0.99,1,196
1204,153,2412,0.99,1,196
1205,153,2413,0.99,1,196
1206,153,2414,0.99,1,196
1207,153,2415,0.99,1,196


In [43]:
%%sql
-- verifying there are no other tracks in album 196
SELECT track_id FROM track
    WHERE album_id = 196

 * sqlite:///chinook.db
Done.


track_id
2406
2407
2408
2409
2410
2411
2412
2413
2414
2415


#### Finalizing the query, adding statistics

In [66]:
%%sql
WITH 
album_tracks AS (
    SELECT album_id, track_id 
    FROM track
    ),

invoice_tracks AS (
    SELECT i.invoice_id, t.track_id
    FROM invoice i
    LEFT JOIN invoice_line il ON il.invoice_id = i.invoice_id
    LEFT JOIN track t ON t.track_id = il.track_id
    ),

invoice_track_album AS (
    SELECT i.invoice_id, MIN(t.album_id) album_id
    FROM invoice i
    LEFT JOIN invoice_line il ON il.invoice_id = i.invoice_id
    LEFT JOIN track t ON t.track_id = il.track_id
    GROUP BY 1  
    ),

invoice_classification AS (
    SELECT i.invoice_id invoice_id,
        CASE WHEN (
            SELECT track_id FROM album_tracks 
                WHERE album_id = ita.album_id 
            EXCEPT
            SELECT track_id FROM invoice_tracks
                WHERE invoice_id = i.invoice_id
            ) IS NULL AND
            (
            SELECT track_id FROM invoice_tracks
                WHERE invoice_id = i.invoice_id
            EXCEPT
            SELECT track_id FROM album_tracks 
                WHERE album_id = ita.album_id 
            ) IS NULL
            THEN 'Y'
            ELSE 'N' 
            END as full_album_purchase_ind
    FROM invoice i
        LEFT JOIN invoice_track_album ita ON ita.invoice_id = i.invoice_id
    )

SELECT 
    CASE full_album_purchase_ind
        WHEN 'Y' THEN 'Full albums'
        ELSE 'Separate tracks'
        END invoice_type,        
    COUNT(invoice_id) invoice_count,
    ROUND(100*COUNT(invoice_id)/itc.invoice_total_count, 2) percentage

    FROM invoice_classification 
        LEFT JOIN
            (SELECT 
             CAST(COUNT(DISTINCT invoice_id) AS FLOAT) invoice_total_count
             FROM invoice) itc
    GROUP BY full_album_purchase_ind


 * sqlite:///chinook.db
Done.


invoice_type,invoice_count,percentage
Separate tracks,500,81.43
Full albums,114,18.57


The table above is the anser to the excersise

#### 5. Questions to play around with
If you'd like more practice, you can look at the schema and come up with some more business questions, and then write queries to answer them. Here are a few to get you started:

- Which artist is used in the most playlists?
- How many tracks have been purchased vs not purchased?
- Is the range of tracks in the store reflective of their sales popularity?
- Do protected vs non-protected media types have an effect on popularity?